# Deep Learning Toolkit for Splunk - Spark MLLib Gradient Boosting Classifier Example

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [ ]:
# this definition exposes all python module imports that should be available in all subsequent commands
import sys
import json
import pandas as pd
import numpy as np
from random import random
from pyspark.sql import SparkSession
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
SparkSession

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| inputlookup dga_domains_features.csv<br>
| eval dga=if(class=="dga",1,0)<br>
| fit MLTKContainer mode=stage algo=spark_gradient_boosting_classifier iterations=10 dga from PC_1 PC_2 PC_3 ut_consonant_ratio ut_digit_ratio ut_domain_length ut_meaning_ratio ut_shannon ut_vowel_ratio into app:dga_spark_gradient_boosting_classifier<br>
| eval class_predicted=if(floor(predicted_dga)==1,"dga","legit")<br>
| table class* *

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [ ]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("dga_spark_gradient_boosting_classifier")
print(df.describe())
print(param)

## Stage 2 - create and initialize a model

In [ ]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    appName = "gradient_boosting_classifer_spark_model"
    if 'options' in param:
        if 'model_name' in param['options']: 
            appName = param['options']['model_name']
    spark = SparkSession\
        .builder\
        .appName(appName)\
        .getOrCreate()
    model['spark'] = spark
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

In [ ]:
model['spark']

In [ ]:
str(model['spark'].sparkContext.getConf().getAll())

## Stage 3 - fit the model

In [ ]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    spark = model['spark']
    sc = spark.sparkContext
    feature_variables = param['feature_variables']
    target_variable = param['target_variables'][0]
    iterations = 10
    if 'options' in param:
        if 'params' in param['options']:
            if 'iterations' in param['options']['params']:
                iterations = int(param['options']['params']['iterations'])
    sdf = spark.createDataFrame(df)
    rdd = sdf.rdd.map(lambda row: LabeledPoint(row[target_variable], [row[x] for x in feature_variables]) ) 

    model['model'] = GradientBoostedTrees.trainClassifier(rdd, categoricalFeaturesInfo={}, numIterations=iterations)

    info = {"message": "model trained"}
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

In [ ]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    spark = model['spark']
    sdf = spark.createDataFrame(df)
    feature_variables = param['feature_variables']
    predictions = model['model'].predict(sdf.rdd.map(lambda row: [row[x] for x in feature_variables]))
    result = pd.DataFrame(predictions.collect())
    return result

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    import shutil
    from pathlib import Path
    if Path(MODEL_DIRECTORY + name).is_dir():
        shutil.rmtree(MODEL_DIRECTORY + name)
    model['model'].save(model['spark'].sparkContext, MODEL_DIRECTORY + name)
    return model

In [ ]:
save(model,'spark_dga_test')

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = init({},{})
    model['model'] = GradientBoostedTreesModel.load(model['spark'].sparkContext, MODEL_DIRECTORY + name)
    return model

In [ ]:
model = load('spark_dga_test')
model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"spark": "no model"}
    if model:
        returns = {"spark_info": str(model['spark'].sparkContext.getConf().getAll()) }
    return returns

In [ ]:
summary(model)

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code